# DECISION AIDING

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import common as cm
import matplotlib.pyplot as plt

This exercise consists of 3 parts. Finish the first part to get a mark of 3.0. The first two parts to get 4.0. Finish all exercies to get 5.0.

# Part 1: Concordance

Given are the following modes of transport: bus, bike, car, train. Each mode is characterized by 2 cost-type criteria: price and time; and 2 gain-type criteria: comfort and reliability. 

Mode of transport | Time | Comfort | Price | Reliability
--- | --- | --- | --- | ---
 **bus**  | 6 | 3  | 6 | 2
 **bike** | 8 | 2  | 2 | 8
 **car**  | 2 | 10 | 9 | 7
 **train**| 3 | 5  | 5 | 6
 **by foot**| 10 | 2  | 0 | 5

In [2]:
data = {
    'mode': ['bus', 'bike', 'car', 'train', 'foot'],
    'time': [6, 8, 2, 3, 10],
    'comfort': [3, 2, 10, 5, 2],
    'price': [6, 2, 9, 5, 0],
    'reliability': [2, 8, 7, 6, 5]
}

criteria = ['time', 'comfort', 'price', 'reliability']

data = pd.DataFrame(data, columns=['mode', 'time', 'comfort', 'price', 'reliability'])
data

,mode,time,comfort,price,reliability
0,bus,6,3,6,2
1,bike,8,2,2,8
2,car,2,10,9,7
3,train,3,5,5,6
4,foot,10,2,0,5


1.2) Below are the parameters, i.e., thresholds, criterion-type, and weights, for each criterion,

In [3]:
parameters = {'time': {'weight': 4, 'q': 1.0, 'p': 2, 'v': 4, 'type': 'cost'},
 'comfort': {'weight': 2, 'q': 2.0, 'p': 3, 'v': 6, 'type': 'gain'},
 'price': {'weight': 3, 'q': 1.0, 'p': 3, 'v': 5, 'type': 'cost'},
 'reliability': {'weight': 1, 'q': 1.5, 'p': 3, 'v': 5, 'type': 'gain'}}

sum_weights = 10.

pd.DataFrame(parameters, columns=['time', 'comfort', 'price','reliability']).reindex(['type', 'q', 'p', 'v', 'weight']).T

,type,q,p,v,weight
time,cost,1.0,2,4,4
comfort,gain,2.0,3,6,2
price,cost,1.0,3,5,3
reliability,gain,1.5,3,5,1


1.3) Finish the below function for calculating a marginal concordance for $i$-th criterion (gain type) $c_i(g_i(A),g_i(B))$ based on q and p thresholds.

In [4]:
def getConcordanceCost(gA, gB, q, p) -> float:
    return getConcordanceGain(gB, gA, q, p)

def getConcordanceGain(gA, gB, q, p) -> float:
    if gA - gB >= -q:
        return 1.0
    elif gA - gB <= -p:
        return 0.0
    else:
        return (p - (gB - gA)) / (p - q)

1.4)  Calculate  comprehensive concordance  index  for  all  criteria  of  alternatives A and B. Remember that comprehensive concordance must be divided by the sum of weights (normalization).  

In [5]:
def getComprehensiveConcordance(A, B, criteria, parameters):
    concordance = 0.0
    # sum_weights = 0.0
    for criterion in criteria:
        parameter = parameters[criterion]
        if parameter['type'] == 'cost':
            concordance += parameter['weight'] * getConcordanceCost(A[criterion], B[criterion], parameter['q'], parameter['p'])
        else:
            concordance += parameter['weight'] * getConcordanceGain(A[criterion], B[criterion], parameter['q'], parameter['p'])
        # sum_weights += parameter['weight']
    
    return concordance / sum_weights

1.5) Check comprehensive concordance indexes for C(bus, some transportation) (HINT: for C(0,1) should be 0.6):

In [6]:
for alternative_id, alternative_row in data.iterrows():
    print("C({0},{1}) = ".format(0, alternative_id), getComprehensiveConcordance(data.loc[0], alternative_row, criteria, parameters))

C(0,0) =  1.0
C(0,1) =  0.6
C(0,2) =  0.3
C(0,3) =  0.5
C(0,4) =  0.6


1.6) Finish the below function for generating a concordance matrix. Use a majority_threshold as a cutting-level. For which pairs the concordance is fulfilled?

In [7]:
def getConcordanceMatrix(data, criteria, parameters, majority_threshold=0.7, do_print: bool = False):
    concordance_matrix = np.zeros((len(data),len(data)))
    if do_print:
        print("The concordance is fulfilled for the following pairs:")
    for A_idx, A_row in data.iterrows():
        for B_idx, B_row in data.iterrows():
            if A_idx == B_idx:
                concordance_matrix[A_idx][B_idx] = 0
                continue
            compr_conc = getComprehensiveConcordance(A_row, B_row, criteria, parameters)
            if compr_conc >= majority_threshold:
                if do_print:
                    print(f'\t{A_row[0]}, {B_row[0]}')
                concordance_matrix[A_idx][B_idx] = 1
            else:
                continue
    if do_print:
        print("(end of fulfilled concordance pairs)")

    return concordance_matrix

In [8]:
print(getConcordanceMatrix(data, criteria, parameters, do_print=True))

The concordance is fulfilled for the following pairs:
	bike, foot
	car, bus
	car, bike
	car, train
	car, foot
	train, bus
	train, car
	train, foot
(end of fulfilled concordance pairs)
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 1. 0. 1. 1.]
 [1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0.]]


HINT :): The resulting matrix should be: <br><br>[[0. 0. 0. 0. 0.]<br>
 [0. 0. 0. 0. 1.]<br>
 [1. 1. 0. 1. 1.]<br>
 [1. 0. 1. 0. 1.]<br>
 [0. 0. 0. 0. 0.]]<br>

# Part 2: outranking graph

2.1) Complete the function for calculating a marginal discordance  $d_i(𝐴,𝐵)$ based on v threshold. $d_i(𝐴,𝐵) = 1$ when A is worse than B on criterion $i$ by at least the veto threshold, zero otherwise.

In [9]:
def getDiscordanceGain(gA, gB, v):
    if gB - gA >= v:
        return 1
    else:
        return 0

def getDiscordanceCost(gA, gB, v):
    return getDiscordanceGain(gB, gA, v)


2.2) Calculate a comprehensive discordance index. $D(a,b) = 1$ if at least one criterion vetoes against aSb.

In [10]:
def getComprehensiveDiscordance(A, B, criteria, parameters): 
    for criterion in criteria:
        parameter = parameters[criterion]
        if parameter['type'] == 'cost':
            if getDiscordanceCost(A[criterion], B[criterion], parameter['v']):
                return 1
        else:
            if getDiscordanceGain(A[criterion], B[criterion], parameter['v']):
                return 1

    return 0

2.3) Check comprehensive discordance indexes for D(bus, some transportation):

In [11]:
for alternative_id, alternative_row in data.iterrows():
    print("D({0},{1}) = ".format(0, alternative_id),getComprehensiveDiscordance(data.loc[0], alternative_row, criteria, parameters))


D(0,0) =  0
D(0,1) =  1
D(0,2) =  1
D(0,3) =  0
D(0,4) =  1


2.4) Finish the below function for calculating a comprehensive discordance matrix.

In [12]:
def getDiscordanceMatrix(data, criteria, parameters):
    discordance_matrix = np.zeros((len(data),len(data)))
    for A_idx, A_row in data.iterrows():
        for B_idx, B_row in data.iterrows():
            if A_idx != B_idx:
                discordance_matrix[A_idx][B_idx] = getComprehensiveDiscordance(A_row, B_row, criteria, parameters)
                continue
                
    return discordance_matrix

In [13]:
getDiscordanceMatrix(data, criteria, parameters)

array([[0., 1., 1., 0., 1.],
       [0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 1., 1., 0.]])

2.5) Now, finish the below function for generating the outranking matrix. This method should take into account both the concordance and discordance matrices.

In [14]:
def getOutrankingMatrix(data, criteria, parameters, majority_threshold):
    concordance_matrix = getConcordanceMatrix(data, criteria, parameters, majority_threshold)
    discordance_matrix = getDiscordanceMatrix(data, criteria, parameters)
    n = len(data)
    outranking_matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if concordance_matrix[i][j] and not discordance_matrix[i][j]:
                outranking_matrix[i][j] = 1
    return outranking_matrix

In [15]:
outranking_matrix = getOutrankingMatrix(data, criteria, parameters, majority_threshold=0.75)
print(outranking_matrix)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]]


2.6) Change outranking matrix to adjacency list (dictionary) of a following form:

In [16]:
def toAdjacencyList(outranking_matrix):
    n = len(outranking_matrix)
    graph = {i:[] for i in range(n)}
    for i in range(n):
        for j in range(n):
            if outranking_matrix[i][j]:
                graph[i].append(j)
    
    return graph

In [17]:
graph = toAdjacencyList(outranking_matrix)
print(graph)

{0: [], 1: [4], 2: [], 3: [0, 2], 4: []}


2.7) Draw outranking graph, and remove cycles (manually).

## REMARK
We've run into the same problem as in one of the previous assignments, we couldn't get Graphviz to work. We still have
the picture though and can manage without generating one ourselves.

In [18]:
cm.PrintGraph(graph, filename="graph_part_2")
plt.imshow(plt.imread("graph_part_2.png"))

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

2.8) **Question: Which mode of transport are in the kernel?**

# Part 3: Kernel

3.1) Given is the below outranking matrix

In [19]:
outranking_matrix = np.array(   [[0., 1., 0., 0., 0., 0., 1., 0.],
                                 [0., 0., 1., 0., 1., 0., 0., 0.],
                                 [0., 0., 0., 0., 0., 0., 0., 1.],
                                 [0., 0., 0., 0., 1., 0., 1., 0.],
                                 [0., 0., 0., 0., 0., 1., 0., 0.],
                                 [0., 0., 0., 0., 0., 0., 1., 0.],
                                 [0., 0., 0., 0., 0., 0., 0., 1.],
                                 [0., 0., 0., 0., 0., 0., 0., 0.]]) 

In [20]:
graph = toAdjacencyList(outranking_matrix)

In [21]:
graph

{0: [1, 6], 1: [2, 4], 2: [7], 3: [4, 6], 4: [5], 5: [6], 6: [7], 7: []}

3.2) Display the outranking graph. **Question: Which alternatives belong to kernel?**

### REMARK - as above in 2.7
The following alternatives belong to the kernel: 0, 2, 3 and 5

In [ ]:
cm.PrintGraph(graph, filename="graph_part_3")
plt.imshow(plt.imread("graph_part_3.png"))

3.3) In this exercise, you are asked to complete the function for constructing a kernel. Firstly, complete the below auxiliary function which reverses edges of the graph (i.e., constructs reversed adjacency dictionary)

In [22]:
def getReverseGraph(graph):
    # n = len(graph)
    reverse_graph = {i:[] for i in graph}
    for i in graph:
        for j in graph[i]:
            reverse_graph[j].append(i)
    
    return reverse_graph

3.4) Verify the correctness: compare the below reverse_graph with the above graph variable.

In [23]:
reverse_graph = getReverseGraph(graph)
print(f'reverse: {reverse_graph}')
print(f'graph:   {graph}')

reverse: {0: [], 1: [0], 2: [1], 3: [], 4: [1, 3], 5: [4], 6: [0, 3, 5], 7: [2, 6]}
graph:   {0: [1, 6], 1: [2, 4], 2: [7], 3: [4, 6], 4: [5], 5: [6], 6: [7], 7: []}


3.2) Now, complete the below function for finding a graph kernel. This algorithm should proceed in the following way: <br>
1) Find non-outranked vertices. Add them to the kernel. <br> 
2) Remove the already found vertices from the graph and these vertices which are surpassed by them.<br>
3) Repeat (go to 1) until all vertices are removed from the graph. 

You can use the auxiliary reverse_graph structure. It can be helpful for finding non-outranked vertices.

In [24]:
def Kernel(graph):
    # REMARK - in this approach, we assume there are no cycles?
    reversed_graph = getReverseGraph(graph)
    kernel = []

    while len(reversed_graph):
        # empty ones go to kernel
        removed_kernel = []
        for key in reversed_graph.keys():
            if not reversed_graph[key]:
                removed_kernel.append(key)
        for key in removed_kernel:
            reversed_graph.pop(key)
        kernel.extend(removed_kernel)

        # remove those that have kernel in their values
        removed = list()
        for key in reversed_graph.keys():
            for val in removed_kernel:
                if val in reversed_graph[key]:
                    removed.append(key)
                    break
        for key in removed:
            reversed_graph.pop(key)

        # remove values of removed from remaining keys
        for key in reversed_graph.keys():
            for val in removed:
                if val in reversed_graph[key]:
                    reversed_graph[key].remove(val)

    return kernel

In [25]:
Kernel(graph)

[0, 3, 2, 5]